<a href="https://colab.research.google.com/github/ioradlin/NAT_project/blob/Encoder/fast_align.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IWSLT

#### data load & prepare

In [13]:
!pip install datasets

In [6]:
from datasets import load_dataset

dataset = load_dataset("bbaaaa/iwslt14-de-en-preprocess")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

' 같은 special token은..

In [10]:
## &apro;, &quot; 등 변환, align 시 단어 수를 encoder input과 동일하게 만들기 위해 미리 tokenize

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import html

def preprocess1(sentence):
  return ' '.join(word_tokenize(html.unescape(sentence)))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [19]:
## fast align input 전처리

def for_fastalign(data):
  for_align= []

  for i, pair in enumerate(data):
    src = preprocess1(pair['de'])
    tgt = preprocess1(pair['en'])
    for_align.append(src+' ||| '+tgt)

  return '\n'.join(for_align)

## fast align

In [ ]:
%cd /content/
!git clone https://github.com/clab/fast_align.git
!sudo apt-get install libgoogle-perftools-dev libsparsehash-dev
%cd fast_align
!mkdir build
%cd build
!cmake ..
!make
%cd /content/

In [28]:
## 빠른 txt 작업

def to_txt(x, name):
  with open(name, 'w') as file:
      file.write(x)

def read_txt(name):
  with open(name, 'r') as file:
      temp= file.read()
  return temp

In [62]:
## ✰✰✰✰✰ .align 읽어서 feritility label로 변환 ✰✰✰✰✰

from collections import Counter

def read_align(name):
  counts= []

  with open(name, 'r') as file:
      for line in file:
          line = line.strip()

          #pairs: 'a list' of 'n-n's
          pairs = line.split()

          #left_numbers: ['0', '0', '2', '1', ...
          left_numbers = [pair.split('-')[0] for pair in pairs]

          #Counter({'14': 5, '3': 4, '2': 3,...
          count = Counter(left_numbers)

          #count: [1, 1, 3, 4, 1, 0, 0, 0, ...
          count= [count.get(str(i), 0) for i in range(max(map(int, count.keys())) + 1)]
          counts.append(count)

          ## count sum check (다르면 fertility label이 될수 없음)
          if len(left_numbers)!=sum(count):
            print('what!?', len(left_numbers),sum(count))
            break

      return counts

#### fertility label 생성

In [69]:
valid= for_fastalign(dataset['validation']['translation'])
to_txt(valid,'valid.txt')

train = for_fastalign(dataset['train']['translation'])
to_txt(train,'train.txt')

test= for_fastalign(dataset['test']['translation'])
to_txt(test,'test.txt')

In [71]:
## 각각 적용 (.txt와 .align 이름만 수정)
!/content/fast_align/build/fast_align -i train.txt -d -o -v > train.align

ARG=i
ARG=d
ARG=o
ARG=v
INITIAL PASS 
.................................................. [50000]
.................................................. [100000]
.................................................. [150000]
..........
expected target length = source length * 1.06981
ITERATION 1
.................................................. [50000]
.................................................. [100000]
.................................................. [150000]
..........
  log_e likelihood: -6.79024e+07
  log_2 likelihood: -9.79625e+07
     cross entropy: 29.8974
        perplexity: 1e+09
      posterior p0: 0.08
 posterior al-feat: -0.169405
       size counts: 2440
ITERATION 2
.................................................. [50000]
.................................................. [100000]
.................................................. [150000]
..........
  log_e likelihood: -1.73326e+07
  log_2 likelihood: -2.50057e+07
     cross entropy: 7.63153
        perplexity: 198.2

기본적으로 final perplexity가 40 정도인데 괜찮은 것인가..?

In [72]:
## fast align을 다 돌렸다면 여기로

test_fertility= read_align('test.align')
valid_fertility= read_align('valid.align')
train_fertility= read_align('train.align')

#### fertility check example

wir -> 1 단어 (we)

durchlaufen -> 2단어 (go through)

initiatiohnsrituale -> 2단어 (initiation rites)

. -> 1단어(.)

In [66]:
test_fertility[4]

[1, 2, 2, 1]

In [67]:
dataset['test']['translation'][4]

{'de': 'wir durchlaufen initiationsrituale .',
 'en': 'we go through initiation rites .'}

In [76]:
len(valid_fertility)==len(dataset['validation']['translation'])

True

#### export (pickle)

In [79]:
import pickle

with open('train_ft.pkl', 'wb') as file: #6.8MB
    pickle.dump(train_fertility, file)

with open('valid_ft.pkl', 'wb') as file: #0.3MB
    pickle.dump(valid_fertility, file)

with open('test_ft.pkl', 'wb') as file:
    pickle.dump(test_fertility, file)

In [81]:
# with open('test_ft.pkl', 'rb') as file:
#     data = pickle.load(file)

# data[:3]